In [2]:
import pandas as pd
import numpy as np
import arcpy

In [8]:
#import importlib.util
#spec = importlib.util.spec_from_file_location("dbconnect", "G:/My Drive/Python/dbconnect.py")
#dbconnect = importlib.util.module_from_spec(spec)
#spec.loader.exec_module(dbconnect)
#connection = dbconnect.sqlconnect()
#wqxmod = importlib.util.spec_from_file_location("wqxsde", "G:\My Drive\Python\Pycharm\wqxsde\wqxsde\sde_tools.py")
#wqxsde = importlib.util.module_from_spec(wqxmod)

In [4]:
schema_file_path = "Domains.xlsx"
char_schema = pd.read_excel(schema_file_path, "CHARACTERISTIC")

In [7]:
wqxpath = "G:/My Drive/Python/Pycharm/wqxsde/"
sys.path.append(wqxpath)
import wqxsde

In [9]:
enviro = "C:/Users/paulinkenbrandt/AppData/Roaming/Esri/Desktop10.6/ArcCatalog/Connection to DEFAULT@uggp.agrc.utah.gov.sde"
conn = arcpy.ArcSDESQLExecute(enviro)
arcpy.env.workspace = enviro
chem_table_name = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results"
activities_table_name ="UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities"
stations_table_name = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Locations"

wqx_results_filename = "ResultsExport.xlsx"
wqx_stations_filename = "MonitoringLocationsExport.xlsx"

# Locations Import and Check

Compare station export from the CDX to the data in the ArcGIS SDE table.

In [ ]:
wqxsde

In [ ]:
wqx_stat_table, sde_stat_table = compare_sde_wqx(wqx_stations_filename,enviro,stations_table_name,table_type="stat")

In [ ]:
sde_stat_import = sde_stat_table[sde_stat_table['inwqx']==0]
sde_stat_import = sde_stat_import.reset_index()
sde_stat_import['TribalLandInd'] = 'No'
sde_stat_import['TribalLandName'] = None

In [ ]:
def get_context(df):
    if pd.isnull(df['USGS_ID']):
        if pd.isnull(df['WIN']):
            if pd.isnull(df['WRNum']):
                df['AltLocationContext'] = None
                df['AltLocationID'] = None
            else:
                df['AltLocationContext'] = 'Utah Water Rights Number'
                df['AltLocationID'] = df['WRNum']
        else:
            df['AltLocationContext'] = 'Utah Well ID'
            df['AltLocationID'] = df['WIN']
    else:
        df['AltLocationContext'] = 'USGS ID'
        df['AltLocationID'] = df['USGS_ID']
    return df

sde_stat_import = sde_stat_import.apply(lambda x: get_context(x),1)
#sde_stat_import[['LocationID','LocationName','AltLocationID','AltLocationContext']]

In [ ]:
import_config_link = "https://cdx.epa.gov/WQXWeb/ImportConfigurationDetail.aspx?mode=import&impcfg_uid=6441"

In [ ]:
stat_col_import_order = ['LocationID', 'LocationName', 'LocationType', 'HUC8', 'HUC12', 
                         'TribalLandInd', 'TribalLandName', 'Latitude', 'Longitude',
                         'HorizontalCollectionMethod', 'HorizontalCoordRefSystem',
                         'State', 'County', 
                         'VerticalMeasure', 'VerticalUnit', 'VerticalCoordRefSystem',
                         'VerticalCollectionMethod',
                         'AltLocationID', 'AltLocationContext',
                         'WellType','WellDepth','WellDepthMeasureUnit', 'AquiferName']
#sde_stat_import = sde_stat_import[sde_stat_import['inwqx']==0]
sde_stat_import1 = sde_stat_import[stat_col_import_order]
sde_stat_import1 = sde_stat_import1[sde_stat_import1['LocationType']!='Barometer']
sde_stat_import1 = sde_stat_import1.sort_values("LocationID")

In [ ]:
sde_stat_import1.to_csv("G:/My Drive/WORK/NGWMN/Database/DataToImport/prepared_csvs/stations_{:%Y%m%d}.csv".format(datetime.datetime.today()),index=False,header=False)

In [ ]:
sde_stat_import1.sort_values("LocationID")

# Chemistry Import and Check

In [13]:
lab_based_chem = "https://cdx.epa.gov/WQXWeb/ImportConfigurationDetail.aspx?impcfg_uid=5926"

field_chem = "https://cdx.epa.gov/WQXWeb/ImportConfigurationDetail.aspx?mode=import&impcfg_uid=5931"

## Get Location ID Matches

In [14]:
enviro = "C:/Users/paulinkenbrandt/AppData/Roaming/Esri/Desktop10.6/ArcCatalog/Connection to DEFAULT@uggp.agrc.utah.gov.sde"
chem_table_name = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results"
activities_table_name ="UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities"
stations_table_name = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Locations"
wqx_results_filename = "G:/My Drive/WORK/NGWMN/Database/ResultsExport.xlsx"
wqx_stations_filename = "G:/My Drive/WORK/Downloads/MonitoringLocationsExport(1).xlsx"



stations = table_to_pandas_dataframe(stations_table_name)
activities = table_to_pandas_dataframe(activities_table_name)


In [15]:
match_spreadsheet_file = "G:/My Drive/WORK/Groundwater Chemistry/UTGS_EDD_20190304.xlsx"
matches = pd.read_excel(match_spreadsheet_file,'UTGS_EDD_20190304')
matches = matches[['Station ID','Sample Number']].drop_duplicates()
matches['Station ID'] = matches['Station ID'].apply(lambda x: "{:.0f}".format(x),1)
matches_dict = matches[['Sample Number','Station ID']].set_index(['Sample Number']).to_dict()['Station ID']


proj_name_matches = {'Arches Monitoring Wells':'UAMW',
                     'Bryce':'UBCW',
                     'Castle Valley':'CAVW',
                     'GSL Chem':'GSLCHEM', 
                     'Juab Valley':'UJVW',
                     'Mills/Mona Wetlands':'MMWET', 
                     'Monroe Septic':'UMSW', 
                     'Ogden Valley':'UOVW', 
                     'Round Valley':'URVH',
                     'Snake Valley':'USVW', 'Snake Valley Wetlands':'SVWET',
                     'Tule Valley Wetlands':'TVWET', 'UGS-NGWMN':'UNGWMN',
                     'WRI - Grouse Creek':'UWRIG', 
                     'WRI - Montezuma':'UWRIM', 
                     'WRI - Tintic Valley':'UWRIT'}

projectmatch = stations[['LocationID','QWNetworkName']].set_index('LocationID').to_dict()['QWNetworkName']

In [16]:
wqx_chem_table, sde_chem_table = compare_sde_wqx(wqx_results_filename,enviro,chem_table_name,table_type="chem")
sde_chem_import = sde_chem_table[sde_chem_table['inwqx']==0]

[1602, 1603, 1604, 1605, 1607, 1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1652, 1653, 1654, 1656, 1657, 1658, 1659, 1660, 1661, 1662, 1663, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1676, 1678, 1679, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695, 1696, 1697, 1698, 1699, 1701, 1702, 1703, 1704, 1705, 1706, 1707, 1708, 1709, 1710, 1711, 1712, 1713, 1714, 1715, 1716, 1717, 1718, 1719, 1720, 1721, 1722, 1723, 1724, 1725, 1726, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1735, 1736, 1737, 1738, 1739, 1740, 1741, 1742, 1743, 1744, 1745, 1746, 1747, 1748, 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773, 1774, 177

In [65]:
state = ProcessStateLabText(file_path,"G:/My Drive/WORK/Groundwater Chemistry",
                            match_spreadsheet_file, schema_file_path)

In [66]:
state.run_calcs()

,ActivityID,MonitoringLocationID,ActivityStartDate,ActivityStartTime,notes,personnel,ResultAnalyticalMethodContext,ResultAnalyticalMethodID,ResultSampleFraction,resultvalue,...,AnalysisStartDate,ResultDetecQuantLimitType,ProjectID,ResultDetectionCondition,CharacteristicName,MethodSpeciation,characteristicgroup,ResultValueType,ResultStatusID,inwqx
0,2136699,415906112485001,2018-07-10,11:00,415906112485001 IRRIGATION WELL #5,EM,USEPA,160.1,Total,996.000,...,2018-07-16,Lower Reporting Limit,UNGWMN,None,Total Dissolved Solids,None,None,Actual,Final,0
1,2136702,415906112485001,2018-07-10,11:00,415906112485001 IRRIGATION WELL #5,EM,USEPA,200.8,Dissolved,1.990,...,2018-07-13,Lower Reporting Limit,UNGWMN,None,As,None,None,Actual,Final,0
2,2136702,415906112485001,2018-07-10,11:00,415906112485001 IRRIGATION WELL #5,EM,USEPA,200.8,Dissolved,78.600,...,2018-07-17,Lower Reporting Limit,UNGWMN,None,Ca,None,None,Actual,Final,0
3,2136702,415906112485001,2018-07-10,11:00,415906112485001 IRRIGATION WELL #5,EM,USEPA,200.8,Dissolved,4.290,...,2018-07-17,Lower Reporting Limit,UNGWMN,None,K,None,None,Actual,Final,0
4,2136702,415906112485001,2018-07-10,11:00,415906112485001 IRRIGATION WELL #5,EM,USEPA,200.8,Dissolved,38.300,...,2018-07-17,Lower Reporting Limit,UNGWMN,None,Mg,None,None,Actual,Final,0
5,2136702,415906112485001,2018-07-10,11:00,415906112485001 IRRIGATION WELL #5,EM,USEPA,200.8,Dissolved,162.000,...,2018-07-17,Lower Reporting Limit,UNGWMN,None,Na,None,None,Actual,Final,0
6,2136702,415906112485001,2018-07-10,11:00,415906112485001 IRRIGATION WELL #5,EM,USEPA,2340B,Dissolved,354.000,...,NaT,Lower Reporting Limit,UNGWMN,None,Hardness,None,None,Actual,Final,0
7,2136700,413810113562001,2018-07-10,13:30,413810113562001 DOMESTIC WELL GROUSE CR,EM,USEPA,160.1,Total,592.000,...,2018-07-16,Lower Reporting Limit,UNGWMN,None,Total Dissolved Solids,None,None,Actual,Final,0
8,2136703,413810113562001,2018-07-10,13:30,413810113562001 DOMESTIC WELL GROUSE CR,EM,USEPA,200.8,Dissolved,2.990,...,2018-07-13,Lower Reporting Limit,UNGWMN,None,As,None,None,Actual,Final,0
9,2136703,413810113562001,2018-07-10,13:30,413810113562001 DOMESTIC WELL GROUSE CR,EM,USEPA,200.8,Dissolved,107.000,...,2018-07-17,Lower Reporting Limit,UNGWMN,None,Ca,None,None,Actual,Final,0


In [64]:
file_path = "G:/My Drive/Python/Pycharm/epa_cdx_wqx_chem_tools/UTGS_EDD_20190304.txt"
match_spreadsheet_file = "G:/My Drive/WORK/Groundwater Chemistry/UTGS_EDD_20190304.xlsx"
schema_file_path = "G:/My Drive/WORK/NGWMN/Database/Domains_And_Schemas/Domains/Domains.xlsx"

class ProcessStateLabText(object):
    
    def __init__(self, file_path, save_path, sample_matches_file, schema_file_path,
                 enviro = "C:/Users/paulinkenbrandt/AppData/Roaming/Esri/Desktop10.6/ArcCatalog/Connection to DEFAULT@uggp.agrc.utah.gov.sde"):
        
        self.chem_table_name = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results"
        self.activities_table_name ="UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities"
        self.stations_table_name = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Locations"
        
        self.save_folder = save_path
        self.schema_file_path = schema_file_path
        self.sample_matches_file = sample_matches_file
        
        self.state_lab_chem = pd.read_csv(file_path,sep="\t")
        
        self.param_explain = {'Fe':'Iron', 'Mn':'Manganese', 'Ca':'Calcium',
                 'Mg':'Magnesium','Na':'Sodium',
                 'K':'Potassium','HCO3':'Bicarbonate', 
                 'CO3':'Carbonate', 'SO4':'Sulfate',
                 'Cl':'Chloride', 'F':'Floride', 'NO3-N':'Nitrate as Nitrogen',
                 'NO3':'Nitrate','B':'Boron','TDS':'Total dissolved solids',
                 'Total Dissolved Solids':'Total dissolved solids',
                 'Hardness':'Total hardness','hard':'Total hardness',
                 'Total Suspended Solids': 'Total suspended solids',
                 'Cond':'Conductivity', 'pH':'pH', 'Cu':'Copper',
                 'Pb':'Lead', 'Zn':'Zinc', 'Li':'Lithium', 'Sr':'Strontium',
                 'Br':'Bromide', 'I':'Iodine', 'PO4':'Phosphate', 'SiO2':'Silica',
                 'Hg':'Mercury', 'NO3+NO2-N':'Nitrate + Nitrite as Nitrogen',
                 'As':'Arsenic', 'Cd':'Cadmium', 'Ag':'Silver',
                 'Alk':'Alkalinity, total', 'P':'Phosphorous',
                 'Ba':'Barium', 'DO':'Dissolved oxygen',
                 'Q':'Discharge', 'Temp':'Temperature',
                 'Hard_CaCO3':'Hardness as Calcium Carbonate', 
                 'DTW':'Depth to water',
                 'O18':'Oxygen-18', '18O':'Oxygen-18', 'D':'Deuterium',
                 'd2H':'Deuterium', 'C14':'Carbon-14',
                 'C14err':'Carbon-14 error', 'Trit_err':'Tritium error',
                 'Meas_Alk':'Alkalinity, total', 'Turb':'Turbidity',
                 'TSS':'Total suspended solids',
                 'C13':'Carbon-13', 'Tritium':'Tritium',
                 'S':'Sulfur', 'density':'density',
                 'Cr':'Chromium', 'Se':'Selenium', 
                 'temp':'Temperature', 'NO2':'Nitrite',
                 'O18err':'Oxygen-18 error', 'd2Herr':'Deuterium error',
                 'NaK':'Sodium + Potassium', 'Al':'Aluminum', 
                 'Be':'Beryllium', 'Co':'Cobalt', 
                 'Mo':'Molydenum', 'Ni':'Nickel', 
                 'V':'Vanadium', 'SAR':'Sodium absorption ratio',
                 'Hard':'Total hardness', 'Free Carbon Dioxide':'Carbon dioxide',
                 'CO2':'Carbon dioxide'
                }
        self.chemcols = {'Sample Number':'ActivityID', 
                'Station ID':'MonitoringLocationID', 
                'Sample Date':'ActivityStartDate',
                'Sample Time':'ActivityStartTime', 
                'Sample Description':'notes', 
                'Collector':'personnel', 
                'Method Agency':'ResultAnalyticalMethodContext',
                'Method ID':'ResultAnalyticalMethodID', 
                'Matrix Description':'ResultSampleFraction', 
                'Result Value':'resultvalue', 
                'Lower Report Limit':'ResultDetecQuantLimitMeasure', 
                'Method Detect Limit':'ResultDetecQuantLimitUnit',
                'Units':'ResultUnit',
                'Analysis Date':'AnalysisStartDate'}
        
        
    def run_calcs(self):
        matches_dict = self.get_sample_matches()
        state_lab_chem = self.state_lab_chem
        state_lab_chem['Station ID'] = state_lab_chem['Sample Number'].apply(lambda x: matches_dict.get(x), 1)
        state_lab_chem['ResultDetecQuantLimitType'] = 'Lower Reporting Limit'
        
        projectmatch = self.get_proj_match()
        state_lab_chem['ProjectID'] = state_lab_chem['Station ID'].apply(lambda x: projectmatch.get(x), 1)
        state_lab_chem['ProjectID'] = state_lab_chem['ProjectID'].apply(lambda x: self.proj_name_matches.get(x),1)
        state_lab_chem['Matrix Description'] = state_lab_chem['Matrix Description'].apply(lambda x: self.ressampfr(x),1)
        state_lab_chem['ResultDetectionCondition'] = state_lab_chem[['Problem Identifier','Result Code']].apply(lambda x: self.lssthn(x),1)
        state_lab_chem['Sample Date'] = pd.to_datetime(state_lab_chem['Sample Date'].str.split(' ',expand=True)[0])
        state_lab_chem['Analysis Date'] = pd.to_datetime(state_lab_chem['Analysis Date'].str.split(' ',expand=True)[0])
        state_lab_chem = state_lab_chem.apply(lambda df: self.renamepar(df),1)
        state_lab_chem = state_lab_chem.rename(columns=self.chemcols)
        chemgroups = self.get_group_names()
        state_lab_chem['characteristicgroup'] = state_lab_chem['CharacteristicName'].apply(lambda x: chemgroups.get(x),1)
        unneeded_cols = ['Trip ID','Agency Bill Code', 
                                       'Test Comment', 'Result Comment', 'Sample Report Limit',
                                       'Chain of Custody','Cost Code', 'Test Number', 
                                       'CAS Number', 'Project Name',
                                       'Sample Received Date', 'Method Description', 'Param Description',
                                       'Dilution Factor','Batch Number','Replicate Number', 
                                       'Sample Detect Limit', 'Problem Identifier', 'Result Code',
                                       'Sample Type', 'Project Comment', 'Sample Comment']
        state_lab_chem = state_lab_chem.drop(unneeded_cols, axis=1)
        state_lab_chem['ResultValueType'] = 'Actual'
        state_lab_chem['ResultStatusID'] = 'Final'
        state_lab_chem['ResultAnalyticalMethodContext'] = state_lab_chem['ResultAnalyticalMethodContext'].apply(lambda x: 'APHA' if x == 'SM' else 'USEPA',1)
        state_lab_chem['inwqx'] = 0
        unitdict = {'MG-L':'mg/l','UG-L':'ug/l','NONE':'None','UMHOS-CM':'uS/cm'}
        state_lab_chem['ResultUnit'] = state_lab_chem['ResultUnit'].apply(lambda x: unitdict.get(x,x),1)
        state_lab_chem['ResultDetecQuantLimitUnit'] = state_lab_chem['ResultUnit']
        self.state_lab_chem = state_lab_chem
        self.save_it(self.save_folder)
        return state_lab_chem
        
    def pull_sde_stations(self):
        stations = table_to_pandas_dataframe(self.stations_table_name, field_names=['LocationID','QWNetworkName'])
        return stations
        
    def get_sample_matches(self):
        matches = pd.read_excel(self.sample_matches_file,'UTGS_EDD_20190304')
        matches = matches[['Station ID','Sample Number']].drop_duplicates()
        matches['Station ID'] = matches['Station ID'].apply(lambda x: "{:.0f}".format(x),1)
        matches_dict = matches[['Sample Number','Station ID']].set_index(['Sample Number']).to_dict()['Station ID']
        return matches_dict

    def get_proj_match(self):
        stations = self.pull_sde_stations()
        self.proj_name_matches = {'Arches Monitoring Wells':'UAMW',
                             'Bryce':'UBCW',
                             'Castle Valley':'CAVW',
                             'GSL Chem':'GSLCHEM', 
                             'Juab Valley':'UJVW',
                             'Mills/Mona Wetlands':'MMWET', 
                             'Monroe Septic':'UMSW', 
                             'Ogden Valley':'UOVW', 
                             'Round Valley':'URVH',
                             'Snake Valley':'USVW', 'Snake Valley Wetlands':'SVWET',
                             'Tule Valley Wetlands':'TVWET', 'UGS-NGWMN':'UNGWMN',
                             'WRI - Grouse Creek':'UWRIG', 
                             'WRI - Montezuma':'UWRIM', 
                             'WRI - Tintic Valley':'UWRIT'}

        projectmatch = stations[['LocationID','QWNetworkName']].set_index('LocationID').to_dict()['QWNetworkName']        

        return projectmatch

    def ressampfr(self, x):
        if str(x).strip() == 'Water, Filtered':
            return 'Dissolved'
        else:
            return 'Total'
    
    def lssthn(self, x):
        if x[0] == '<':
            return "Below Reporting Limit"
        elif x[0] == '>':
            return "Above Operating Range"
        elif x[1] == 'U' and pd.isna(x[0]):
            return "Not Detected"
        else:
            return None
    
    def renamepar(self, df):
        
        x = df['Param Description']
        x = str(x).strip()
        y = None

        if x in self.param_explain.keys():
            z = self.param_explain.get(x)

        if " as " in x:
            z = x.split(' as ')[0]
            y = x.split(' as ')[1]
        else:
            z = x

        if str(z).strip() == 'Alkalinity':
            z = 'Alkalinity, total'

        if y == 'Calcium Carbonate':
            y = 'as CaCO3'
        elif y == 'Carbonate':
            y = 'as CO3'
        elif y == 'Nitrogen':
            y = 'as N'
        elif z == 'Total Phosphate' and pd.isna(y):
            z = 'Orthophosphate'
            y = 'as PO4'
        df['CharacteristicName'] = z
        df['MethodSpeciation'] = y
        return df

    def check_chems(self, df, char_schema):
        missing_chem = []
        for chem in df['CharacteristicName'].unique():
            if chem not in char_schema['Name'].values:
                print(chem)
                missing_chem.append(chem)
        return missing_chem
        
    def get_group_names(self):
        char_schema = pd.read_excel(self.schema_file_path, "CHARACTERISTIC")
        chemgroups = char_schema[['Name','Group Name']].set_index(['Name']).to_dict()['Group Name']
        return chemgroups

    def save_it(self,savefolder):
        self.state_lab_chem.to_csv("{:}/state_lab_to_sde_{:%Y%m%d}.csv".format(savefolder, pd.datetime.today()))

In [18]:
state_lab_chem['AnalysisStartDate'] = state_lab_chem['AnalysisStartDate'].apply(lambda x: None if x is pd.NaT else "{:%Y-%m-%d}".format(x),1)

In [19]:
connection = dbconnect.sqlconnect()

In [22]:
enviro = "C:/Users/paulinkenbrandt/AppData/Roaming/Esri/Desktop10.6/ArcCatalog/UGS_SDE.sde"
act_table = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities"
fieldnames=['ActivityID','ProjectID','MonitoringLocationID','ActivityStartDate',
              'ActivityStartTime','notes','personnel','created_user','created_date','last_edited_user','last_edited_date']
labfields = ['ActivityID', 'MonitoringLocationID', 
             'ResultAnalyticalMethodContext', 'ResultAnalyticalMethodID',
             'ResultSampleFraction', 'resultvalue', 'ResultDetecQuantLimitMeasure',
             'ResultDetecQuantLimitUnit', 'ResultUnit', 'AnalysisStartDate', 
             'ResultDetecQuantLimitType', 'ResultDetectionCondition',
             'CharacteristicName', 'MethodSpeciation', 'characteristicgroup',  
             'ResultValueType', 'ResultStatusID']

In [24]:


for feld in labfields:
    if feld not in sde_chem_table.columns:
        print(feld)
        

In [52]:
def edit_table(df, fieldnames = None,
               sde_table="UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results",
               enviro="C:/Users/paulinkenbrandt/AppData/Roaming/Esri/Desktop10.6/ArcCatalog/UGS_SDE.sde"):    
       
    arcpy.env.workspace = enviro
    
    if len(fieldnames)>0:
        pass
    else:
        fieldnames = df.columns
    
    read_descr = arcpy.Describe(res_table)
    sde_field_names = []
    for field in read_descr.fields:
        sde_field_names.append(field.name)
    sde_field_names.remove('OBJECTID')

    for name in fieldnames:
        if name not in sde_field_names:
            fieldnames.remove(name)
            printmes("{:} not in {:} fieldnames!".format(name, sde_table))
            
    try:    
        egdb_conn = arcpy.ArcSDESQLExecute(enviro)
        egdb_conn.startTransaction()
        print("Transaction started...")
        # Perform the update
        try:
            sqlid = """SELECT max(OBJECTID) FROM {:};""".format(sde_table)

            objid = egdb_conn.execute(sqlid)
            
            subset = df[fieldnames]
            rowlist = subset.values.tolist()
            
            sqlbeg = "INSERT INTO {:}({:},OBJECTID)\nVALUES ".format(sde_table, ",".join(map(str, fieldnames)))
            sqlendlist = []
            
            for j in range(len(rowlist)):
                objid += 1
                strfill = ["("]
                #sqlendlist.append("('{:}',{:})".format("','".join(map(str,rowlist[j])),objid))
                for k in range(len(rowlist[j])):
                    if pd.isna(rowlist[j][k]):
                        strfill.append("NULL")
                    elif isinstance(rowlist[j][k],(int,float)):
                        strfill.append("{:}".format(rowlist[j][k]))
                    else:
                        strfill.append("'{:}'".format(rowlist[j][k]))
                        
                strfill.append(",{:})".format(objid))
                sqlendlist.append(",".join(map(str, strfill)))               

            sqlend = "{:}".format(",".join(sqlendlist))
            sql = sqlbeg + sqlend    
            
            egdb_return = egdb_conn.execute(sql)
                
            # If the update completed successfully, commit the changes.  If not, rollback.
            if egdb_return == True:
                egdb_conn.commitTransaction()
                print("Committed Transaction")
            else:
                egdb_conn.rollbackTransaction()
                print("Rolled back any changes.")
                print("++++++++++++++++++++++++++++++++++++++++\n")
        except Exception as err:
            print(err)
            egdb_return = False
    # Disconnect and exit
        del egdb_conn
    except Exception as err:
        print(err)

In [33]:
try:
    df = state_lab_chem
    sdeact = table_to_pandas_dataframe(act_table)
    df = df[~df['ActivityID'].isin(sdeact['ActivityID'])]
    df['created_user'] = 'paulinkenbrandt'
    df['last_edited_user'] = 'paulinkenbrandt'
    df['created_date'] = pd.datetime.today()
    df['last_edited_date'] = pd.datetime.today()
    
    fieldnames=['ActivityID','ProjectID','MonitoringLocationID','ActivityStartDate',
              'ActivityStartTime','notes','personnel','created_user','created_date','last_edited_user','last_edited_date']
    
    for i in range(0,len(df),500):
        j = i + 500
        if j > len(df):
            j = len(df)
        subset = df.iloc[i:j]
        print("{:} to {:} complete".format(i,j))
        edit_table(subset, activities)
        print('success!')
except Exception as err:
    print(err)
    print('fail!')
    pass

0 to 500 complete
success!
500 to 1000 complete
success!
1000 to 1500 complete
success!
1500 to 1503 complete
success!


In [51]:
try:
    df = state_lab_chem
    chem_table_name = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results"
    sdechem = table_to_pandas_dataframe(chem_table_name)
    df = df[~df['ActivityID'].isin(sdechem['ActivityID'])]
    df['created_user'] = 'paulinkenbrandt'
    df['last_edited_user'] = 'paulinkenbrandt'
    df['created_date'] = pd.datetime.today()
    df['last_edited_date'] = pd.datetime.today()

    fieldnames = ['ActivityID', 'MonitoringLocationID', 'ResultAnalyticalMethodContext', 'ResultAnalyticalMethodID',
                  'ResultSampleFraction', 
                  'resultvalue', 'ResultDetecQuantLimitMeasure', 'ResultDetecQuantLimitUnit', 'ResultUnit', 
                  'AnalysisStartDate', 'ResultDetecQuantLimitType', 'ResultDetectionCondition','CharacteristicName', 
                  'MethodSpeciation', 'characteristicgroup', 'ResultValueType', 'ResultStatusID',
                  'inwqx','created_user','last_edited_user','created_date','last_edited_date']
    
    for i in range(0,len(df),500):
        j = i + 500
        if j > len(df):
            j = len(df)
        subset = df.iloc[i:j]
        print("{:} to {:} complete".format(i,j))
        edit_table(subset, fieldnames)
        print('success!')
except Exception as err:
    print(err)
    print('fail!')
    pass


0 to 500 complete
name 'res_table' is not defined
fail!


In [ ]:
stations[['LocationName','AltLocationID','Offset','BaroLoggerType']][stations['AltLocationID']>1000].to_clipboard()

In [ ]:
CREATE VIEW phy_chem_to_wqx
AS
SELECT 
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityID, 
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ProjectID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.MonitoringLocationID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityType,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityMediaName,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityStartDate,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityStartTime,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityStartTimeZone,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.Organization,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.SampleCollMethodID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.SampleCollEquipmentName,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetectionCondition,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.CharacteristicName,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.MethodSpeciation,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultSampleFraction,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultValue,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultUnit,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultQualifier,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultStatusID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultValueType,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultAnalyticalMethodID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultAnalyticalMethodContext,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.LaboratoryName,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.AnalysisStartDate,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.AnalysisStartTime,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.AnalysisStartTimeZone,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetecQuantLimitType,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetecQuantLimitMeasure,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetecQuantLimitUnit

FROM
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results 
JOIN
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities ON
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityID = UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ActivityID